In [78]:
import numpy as np
import time
import datetime
import json
import pandas as pd
import time

import mysql.connector
from mysql.connector import Error

from bokeh.plotting import *
from bokeh.layouts import gridplot
from bokeh.models import (Span, ColumnDataSource, LogColorMapper, ColorMapper, LogTicker, ColorBar, BasicTicker, 
                          LinearColorMapper, PrintfTickFormatter, HoverTool, CategoricalColorMapper, Range1d)
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import show, output_notebook
from bokeh.models.glyphs import Text
import bokeh.palettes as bp

# import hail as hl

import urllib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

In [2]:
json_url = urllib.request.urlopen('https://api.helioviewer.org/?action=getDataSources')
hv_keys = json.loads(json_url.read())

In [61]:
obs = 'SDO'
instr = 'AIA'
wav = '94'
sId = 'sourceId'

In [62]:
start_time=time.time()
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='hv',
                                         user='hv_varun',
                                         password='Helioviewer@2020')

    sql_select_Query = "SELECT filepath, date, sourceid FROM data WHERE sourceId=%s;"%hv_keys[obs][instr][wav][sId]
#     sql_select_Query = "SELECT count(*) FROM data WHERE filepath LIKE '/AIA/1600/%';"
#     sql_select_Query = "SELECT * FROM data LIMIT 20;"
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
    print("Total number of rows in data is: ", cursor.rowcount)

    print("\nPrinting each laptop record")
#     for row in records:
#         print("Index = ", row[0])
#         print("Location = ", row[1])
#         print("Filename  = ", row[2])
#         print("OBS_DATE  = ", row[3], "\n")

except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
        print("MySQL connection is closed", time.time()-start_time )

Total number of rows in data is:  8729086

Printing each laptop record
MySQL connection is closed 127.1820752620697


In [63]:
hv = pd.DataFrame(records, columns=cursor.column_names)
hv = hv.sort_values('date').reset_index(drop=True)
hv

,filepath,date,sourceid
0,/AIA/94/2010/06/02,2010-06-02 00:05:33,8
1,/AIA/94/2010/06/02,2010-06-02 00:05:45,8
2,/AIA/94/2010/06/02,2010-06-02 00:05:57,8
3,/AIA/94/2010/06/02,2010-06-02 00:06:21,8
4,/AIA/94/2010/06/18,2010-06-18 00:00:20,8
...,...,...,...
8729081,/AIA/2020/08/03/94,2020-08-03 14:08:47,8
8729082,/AIA/2020/08/03/94,2020-08-03 14:09:23,8
8729083,/AIA/2020/08/03/94,2020-08-03 14:09:59,8
8729084,/AIA/2020/08/03/94,2020-08-03 14:10:35,8


# Coverage

In [ ]:
day_bins = pd.date_range(hv['date'].min().replace(day=1), (hv['date'].max() + pd.DateOffset(months=1)).replace(day=1) , freq='D').to_period('D').to_timestamp()
day_bins

In [ ]:
counts, bins = np.histogram(hv['date'], bins = day_bins)#.size+1, range=(day_bins[0], day_bins[-1]))

In [ ]:
month_bins = pd.date_range(day_bins.min(), day_bins.max(), freq='M').to_period('M').to_timestamp()
month_bins

In [ ]:
n_days = 31
n_months = month_bins.size # ((day_bins[-1].year-day_bins[0].year)*12 + (day_bins[-1].month-day_bins[0].month)+1)
year_month = month_bins.year.astype(str) + ' ' + month_bins.month_name()# (year.year+year.month/12)#.astype(str)
hv_cov = pd.DataFrame(data=[[np.nan]*n_days]*n_months, index = year_month, columns=np.arange(1,32).astype(str))
hv_cov.index.name='Year'
hv_cov.columns.name = 'Date'
hv_cov

In [ ]:
idx_month_end = pd.Series(day_bins)[pd.Series(day_bins).dt.is_month_start].index
idx_month_end = idx_month_end.union(pd.Index([0]))
idx_month_end

In [ ]:
counts[(day_bins[:-1]<hv['date'].min()-pd.DateOffset(days=1)) | (day_bins[:-1]>hv['date'].max())] = -1

In [ ]:
tot=0
for i in range(idx_month_end.size-1):
    hv_cov.iloc[i][:idx_month_end[i+1]-idx_month_end[i]] = counts[idx_month_end[i]:idx_month_end[i+1]]
hv_cov[hv_cov<0] = np.nan

In [ ]:
hv_cov

In [ ]:
df = pd.DataFrame(hv_cov.stack(dropna=False), columns=['coverage']).reset_index()
df['Year']=df['Year'].astype(str)

years = hv_cov.index.values#.astype(str)
dates = hv_cov.columns.values.astype(str)

colors = bp.Viridis[256]# ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=np.nanmin(df.coverage), high=np.nanmax(df.coverage))

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

# output_file('AIA1600_coverage.html')
p = figure(title="AIA 1600 Coverage ({0} - {1})".format(years[0], years[-1]),
#            x_axis_type='datetime',
           x_range=years, y_range=list(reversed(dates)),
           x_axis_location="above", plot_width=1400, plot_height=800,
           x_axis_label="Year Month", y_axis_label="Date",
           tooltips=[('Date', '@Year @Date'), ('#Data Files', '@coverage')],
           tools=TOOLS, toolbar_location='below')

# p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi / 3
p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label_text_font_size = "12pt"
p.xaxis.visible = True
p.xgrid.visible = True

p.xaxis.major_label_text_font_size = "7pt"
p.yaxis.major_label_text_font_size = "8pt"


p.rect(x="Year", y="Date", width=1, height=1,
       source=df,
       fill_color={'field': 'coverage', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10px",
                     ticker=BasicTicker(desired_num_ticks=10),
                     formatter=PrintfTickFormatter(format="%d"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')
output_file('AIA94_coverage.html')
show(p)

# Histogram

In [147]:
def interactive_histogram(df, n_bins, title, x_axis_label, x_tooltip, log=None):

    arr_hist, edges = np.histogram(df, bins=n_bins)

    # Column data source
    df_hist = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
    total = df_hist['count'].sum()
    df_hist['f_count'] = ['%d' % count for count in df_hist['count']]
    df_hist['f_percent'] = ['%.2f%%' %(count/total*100) for count in df_hist['count']]
    df_hist['f_interval'] = ['%d to %d ' % (left, right) for left, right in zip(df_hist['left'], df_hist['right'])]
    # column data source
    hist_src = ColumnDataSource(df_hist)
    
    #cumulative data
    cumulative_data = np.cumsum(arr_hist)
    x_bins = np.arange(0,df.max(),100)[1:]
    df_cum = pd.DataFrame({'count_cum': cumulative_data, 'x': x_bins})
    cum_src = ColumnDataSource(df_cum)
#     df_hist['f_count'] = np.log10(df_hist['f_count']+1)
    # Set up the figure same as before
    panels = []
    for axis_type in ["linear", "log"]:
        p = figure(y_axis_type = axis_type,
                   title = title,
                   x_axis_label = x_axis_label, 
                   y_axis_label = 'Day count',
                   background_fill_color="#fafafa",
                   y_range = (0.9,df_hist['count'].max() + 100))
        # Add a quad glyph with source this time
        p.quad(bottom=0.9,
               top='count', 
               left='left', 
               right='right', 
               source=hist_src,
               fill_color='navy',
               alpha=0.5,
#                hover_fill_color='navy',
               hover_fill_alpha=0.2,
               line_color='white',
               legend='Histogram')
#         p.y_range(Range1d(0.8,df_hist['count'].max()))
        # Add style to the plot
        p.title.align = 'center'
        p.title.text_font_size = '18pt'
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'
        p.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        p.legend.location = "top_left"
#         p.grid.grid_line_color="white"
        
        text_source = ColumnDataSource(dict(x=[x_bins.max()*3/4],y=[df_hist['count'].max()*3/4],text=['Total Day Count = \n %d'%total]))
        glyph = Text(x="x", y="y", text="text", text_color="black")
        p.add_glyph(text_source, glyph)
        # Add a hover tool referring to the formatted columns
        hover = HoverTool(tooltips = [(x_tooltip, '@f_interval'),
                                      ('Day count', '@f_count'),
                                      ('Day count percentage', '@f_percent')])

        # Add the hover tool to the graph
        p.add_tools(hover)

        
        p2 = figure(y_axis_type=axis_type,
                   title = title,
                   x_axis_label = x_axis_label, 
                   y_axis_label = 'Day count',
                   background_fill_color="#fafafa")
        
        p2.line(x='x', y='count_cum', line_color='#036564', line_width=3, source=cum_src, hover_line_alpha=0.5, legend="Cumulative distribution")
        p2.circle(x='x', y='count_cum', line_color='#036564', line_width=5, source=cum_src, legend="Cumulative distribution" )
        hover = HoverTool(line_policy='nearest', tooltips = [(x_tooltip, '@x'), ('Cumulative Day count', '@count_cum')])
        
        p2.add_layout(Span(location=1800, dimension='height'))#, legend='Expected date file count'))
        # Add the hover tool to the graph
        p2.add_tools(hover)
        p2.title.align = 'center'
        p2.title.text_font_size = '18pt'
        p2.xaxis.axis_label_text_font_size = '12pt'
        p2.xaxis.major_label_text_font_size = '12pt'
        p2.yaxis.axis_label_text_font_size = '12pt'
        p2.yaxis.major_label_text_font_size = '12pt'
        p2.legend.location = "top_left"
        
        
        grid = gridplot([[p, p2]])#, plot_width=250, plot_height=250)
        panel = Panel(child=grid, title=axis_type)
        panels.append(panel)
    tabs = Tabs(tabs=panels)

    return tabs

In [125]:
day_bins = pd.date_range(hv['date'].min().floor('D'), hv['date'].max()+pd.DateOffset(days=1), freq='D').to_period('D').to_timestamp()

In [126]:
day_bins

DatetimeIndex(['2010-06-02', '2010-06-03', '2010-06-04', '2010-06-05',
               '2010-06-06', '2010-06-07', '2010-06-08', '2010-06-09',
               '2010-06-10', '2010-06-11',
               ...
               '2020-07-26', '2020-07-27', '2020-07-28', '2020-07-29',
               '2020-07-30', '2020-07-31', '2020-08-01', '2020-08-02',
               '2020-08-03', '2020-08-04'],
              dtype='datetime64[ns]', length=3717, freq='D')

In [127]:
counts, bins = np.histogram(hv['date'], bins = day_bins)#.size+1, range=(day_bins[0], day_bins[-1]))
# hist, edges = np.histogram(counts, bins=range(0,counts.max(),100))

In [148]:
output_file('AIA94_histogram.html')

df = counts
n_bins = range(0,counts.max(),100)
title = 'Histogram for AIA 94'
x_axis_label = 'No. of Data files'
x_tooltip = '#Data files'
btabs = interactive_histogram(df,n_bins,title,x_axis_label,x_tooltip,log='log')
show(btabs)